In [14]:
# %%
import os
import requests
import zipfile
import pandas as pd
import datetime as dt
import yfinance as  yf

In [25]:
base_url = 'https://www.cftc.gov/files/dea/history/fut_disagg_txt_{}.zip'
output_dir = os.path.join(os.getcwd(), 'data')
output_file = 'fut_disagg.txt'
yf_code = 'ZW=F'
start_date = '2020-01-01'
end_date = '2021-01-01'
cftc_market_code = '001602'

In [26]:
for year in range(2021, 2020, -1):
    print(f"Downloading CFTC data for year {year}...")

    # Construct the URL and download path for this year
    url = base_url.format(year)
    output_zip = os.path.join(output_dir, f'fut_disagg_txt_{year}.zip')

    # Download the file
    r = requests.get(url)

    # Save it as a binary file
    with open(output_zip, 'wb') as f:
        f.write(r.content)

    # Open the downloaded zip file
    with zipfile.ZipFile(output_zip, 'r') as zip_ref:
        # Extract all the contents into the data directory
        zip_ref.extractall(output_dir)

    # The zip file is now unzipped. You can remove the zip file if you wish:
    os.remove(output_zip)

    # Load the data from the extracted file
    new_data = pd.read_csv(os.path.join(output_dir, f'f_year.txt'), delimiter='\t')

    # Append the data to the output file
    if os.path.exists(os.path.join(output_dir,output_file)):
        new_data.to_csv(os.path.join(output_dir,output_file), mode='a', header=False, index=False)
    else:
        new_data.to_csv(os.path.join(output_dir,output_file), mode='w', header=True, index=False)

In [17]:

    
print("Downloaded, selecting data...")
    
yf_df = yf.download(yf_code, start = start_date, end = end_date, progress = False)
yf_df


Downloaded, selecting data...


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2020-01-02,559.50,567.50,558.25,560.25,560.25,49931
2020-01-03,560.75,562.00,551.75,554.50,554.50,44180
2020-01-06,552.50,555.25,546.75,550.00,550.00,47904
2020-01-07,549.50,551.00,545.50,550.25,550.25,41346
2020-01-08,549.50,555.75,545.25,552.75,552.75,51322
...,...,...,...,...,...,...
2020-12-24,631.00,633.75,623.50,627.00,627.00,35656
2020-12-28,627.00,631.50,607.00,614.25,614.25,53532
2020-12-29,614.25,623.25,611.50,618.50,618.50,35420


In [31]:
print("Output_file: ", output_file)
cftc_df = pd.read_csv(os.path.join(output_dir, output_file), delimiter=',')
print("Col:", cftc_df.columns)
cftc_df


Output_file:  fut_disagg.txt
Col: Index(['Market_and_Exchange_Names,"As_of_Date_In_Form_YYMMDD","Report_Date_as_YYYY-MM-DD","CFTC_Contract_Market_Code","CFTC_Market_Code","CFTC_Region_Code","CFTC_Commodity_Code","Open_Interest_All","Prod_Merc_Positions_Long_All","Prod_Merc_Positions_Short_All","Swap_Positions_Long_All","Swap__Positions_Short_All","Swap__Positions_Spread_All","M_Money_Positions_Long_All","M_Money_Positions_Short_All","M_Money_Positions_Spread_All","Other_Rept_Positions_Long_All","Other_Rept_Positions_Short_All","Other_Rept_Positions_Spread_All","Tot_Rept_Positions_Long_All","Tot_Rept_Positions_Short_All","NonRept_Positions_Long_All","NonRept_Positions_Short_All","Open_Interest_Old","Prod_Merc_Positions_Long_Old","Prod_Merc_Positions_Short_Old","Swap_Positions_Long_Old","Swap__Positions_Short_Old","Swap__Positions_Spread_Old","M_Money_Positions_Long_Old","M_Money_Positions_Short_Old","M_Money_Positions_Spread_Old","Other_Rept_Positions_Long_Old","Other_Rept_Positions_Sho

Market_and_Exchange_Names,"As_of_Date_In_Form_YYMMDD","Report_Date_as_YYYY-MM-DD","CFTC_Contract_Market_Code","CFTC_Market_Code","CFTC_Region_Code","CFTC_Commodity_Code","Open_Interest_All","Prod_Merc_Positions_Long_All","Prod_Merc_Positions_Short_All","Swap_Positions_Long_All","Swap__Positions_Short_All","Swap__Positions_Spread_All","M_Money_Positions_Long_All","M_Money_Positions_Short_All","M_Money_Positions_Spread_All","Other_Rept_Positions_Long_All","Other_Rept_Positions_Short_All","Other_Rept_Positions_Spread_All","Tot_Rept_Positions_Long_All","Tot_Rept_Positions_Short_All","NonRept_Positions_Long_All","NonRept_Positions_Short_All","Open_Interest_Old","Prod_Merc_Positions_Long_Old","Prod_Merc_Positions_Short_Old","Swap_Positions_Long_Old","Swap__Positions_Short_Old","Swap__Positions_Spread_Old","M_Money_Positions_Long_Old","M_Money_Positions_Short_Old","M_Money_Positions_Spread_Old","Other_Rept_Positions_Long_Old","Other_Rept_Positions_Short_Old","Other_Rept_Positions_Spread_Old","Tot_Rept_Positions_Long_Old","Tot_Rept_Positions_Short_Old","NonRept_Positions_Long_Old","NonRept_Positions_Short_Old","Open_Interest_Other","Prod_Merc_Positions_Long_Other","Prod_Merc_Positions_Short_Other","Swap_Positions_Long_Other","Swap__Positions_Short_Other","Swap__Positions_Spread_Other","M_Money_Positions_Long_Other","M_Money_Positions_Short_Other","M_Money_Positions_Spread_Other","Other_Rept_Positions_Long_Other","Other_Rept_Positions_Short_Other","Other_Rept_Positions_Spread_Other","Tot_Rept_Positions_Long_Other","Tot_Rept_Positions_Short_Other","NonRept_Positions_Long_Other","NonRept_Positions_Short_Other","Change_in_Open_Interest_All","Change_in_Prod_Merc_Long_All","Change_in_Prod_Merc_Short_All","Change_in_Swap_Long_All","Change_in_Swap_Short_All","Change_in_Swap_Spread_All","Change_in_M_Money_Long_All","Change_in_M_Money_Short_All","Change_in_M_Money_Spread_All","Change_in_Other_Rept_Long_All","Change_in_Other_Rept_Short_All","Change_in_Other_Rept_Spread_All","Change_in_Tot_Rept_Long_All","Change_in_Tot_Rept_Short_All","Change_in_NonRept_Long_All","Change_in_NonRept_Short_All","Pct_of_Open_Interest_All","Pct_of_OI_Prod_Merc_Long_All","Pct_of_OI_Prod_Merc_Short_All","Pct_of_OI_Swap_Long_All","Pct_of_OI_Swap_Short_All","Pct_of_OI_Swap_Spread_All","Pct_of_OI_M_Money_Long_All","Pct_of_OI_M_Money_Short_All","Pct_of_OI_M_Money_Spread_All","Pct_of_OI_Other_Rept_Long_All","Pct_of_OI_Other_Rept_Short_All","Pct_of_OI_Other_Rept_Spread_All","Pct_of_OI_Tot_Rept_Long_All","Pct_of_OI_Tot_Rept_Short_All","Pct_of_OI_NonRept_Long_All","Pct_of_OI_NonRept_Short_All","Pct_of_Open_Interest_Old","Pct_of_OI_Prod_Merc_Long_Old","Pct_of_OI_Prod_Merc_Short_Old","Pct_of_OI_Swap_Long_Old","Pct_of_OI_Swap_Short_Old","Pct_of_OI_Swap_Spread_Old","Pct_of_OI_M_Money_Long_Old","Pct_of_OI_M_Money_Short_Old","Pct_of_OI_M_Money_Spread_Old","Pct_of_OI_Other_Rept_Long_Old","Pct_of_OI_Other_Rept_Short_Old","Pct_of_OI_Other_Rept_Spread_Old","Pct_of_OI_Tot_Rept_Long_Old","Pct_of_OI_Tot_Rept_Short_Old","Pct_of_OI_NonRept_Long_Old","Pct_of_OI_NonRept_Short_Old","Pct_of_Open_Interest_Other","Pct_of_OI_Prod_Merc_Long_Other","Pct_of_OI_Prod_Merc_Short_Other","Pct_of_OI_Swap_Long_Other","Pct_of_OI_Swap_Short_Other","Pct_of_OI_Swap_Spread_Other","Pct_of_OI_M_Money_Long_Other","Pct_of_OI_M_Money_Short_Other","Pct_of_OI_M_Money_Spread_Other","Pct_of_OI_Other_Rept_Long_Other","Pct_of_OI_Other_Rept_Short_Other","Pct_of_OI_Other_Rept_Spread_Other","Pct_of_OI_Tot_Rept_Long_Other","Pct_of_OI_Tot_Rept_Short_Other","Pct_of_OI_NonRept_Long_Other","Pct_of_OI_NonRept_Short_Other","Traders_Tot_All","Traders_Prod_Merc_Long_All","Traders_Prod_Merc_Short_All","Traders_Swap_Long_All","Traders_Swap_Short_All","Traders_Swap_Spread_All","Traders_M_Money_Long_All","Traders_M_Money_Short_All","Traders_M_Money_Spread_All","Traders_Other_Rept_Long_All","Traders_Other_Rept_Short_All","Traders_Other_Rept_Spread_All","Traders_Tot_Rept_Long_All","Traders_Tot_Rept_Short_All","Traders_Tot_Old","Traders_Prod_

In [19]:
df = pd.merge(yf_df, cftc_df, left_index=True, right_on='Report_Date_as_YYYY-MM-DD', how='outer').fillna(method='bfill')
df.set_index('Date', inplace=True)
df

KeyError: 'Report_Date_as_YYYY-MM-DD'